In [1]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
import glob
from keras.models import Sequential
from keras.layers import Dense,Conv1D,Conv2D,MaxPooling2D,Flatten,AveragePooling2D,Input,BatchNormalization,AveragePooling1D
from keras.optimizers import RMSprop

from utils.dataloader import read_edf_to_raw, calculate_neg_pos

2022-07-06 12:21:55.119325: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/guisoares/catkin_ws/devel/lib:/opt/ros/noetic/lib:/usr/lib/x86_64-linux-gnu/:/home/guisoares/catkin_ws/install/lib/mavlink_sitl_gazebo/plugins
2022-07-06 12:21:55.119353: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def create_model_article():
    # Model creation
    model = Sequential()
    # model.add(Input(shape=(23,25600,1)))

    model.add(Conv1D(4, 5, activation='relu', input_shape=(12800, 23), name='conv1'))
    model.add(BatchNormalization(name='batch1'))
    model.add(AveragePooling1D(pool_size=2, strides=2, name='pool1'))
    # output ~(12800,4)

    model.add(Conv1D(4, 6, activation='relu', name='conv2'))
    model.add(BatchNormalization(name='batch2'))
    model.add(AveragePooling1D(pool_size=4, strides=1,name='pool2'))
    # output ~(12800,4)

    model.add(Conv1D(10, 4, activation='relu', name='conv3'))
    model.add(BatchNormalization(name='batch3'))
    model.add(AveragePooling1D(pool_size=2, strides=2, name='pool3'))
    # output ~(6400,4)

    model.add(Conv1D(10, 4, activation='relu', name='conv4'))
    model.add(BatchNormalization(name='batch4'))
    model.add(AveragePooling1D(pool_size=2, strides=4, name='pool4'))
    # output ~(1200,4)

    model.add(Conv1D(15, 2, activation='relu', name='conv5'))
    model.add(BatchNormalization(name='batch5'))
    model.add(AveragePooling1D(pool_size=2, strides=4, name='pool5'))

    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    return model

In [3]:
def create_model_customized():
    # Model creation
    model = Sequential()
    # model.add(Input(shape=(23,25600,1)))

    model.add(Conv2D(16, 3, activation='relu', padding='same', input_shape=(25600, 23, 1), name='conv1'))
    model.add(BatchNormalization(name='batch1'))
    model.add(MaxPooling2D(pool_size=(32,2), name='pool1'))

    model.add(Conv2D(8, 5, activation='relu', padding='same', name='conv2'))
    model.add(BatchNormalization(name='batch2'))
    model.add(MaxPooling2D(pool_size=(16,2), name='pool2'))

    model.add(Conv2D(4, 7, activation='relu', padding='same', name='conv3'))
    model.add(BatchNormalization(name='batch3'))
    model.add(MaxPooling2D(pool_size=(8,2), name='pool3'))

    model.add(Conv2D(2, 3, activation='relu',  padding='same', name='conv7'))
    model.add(BatchNormalization(name='batch7'))
    model.add(MaxPooling2D(pool_size=(4,2), name='pool4'))

    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    return model

In [4]:
model = create_model_article()
model.summary()

2022-07-06 12:21:58.224551: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/guisoares/catkin_ws/devel/lib:/opt/ros/noetic/lib:/usr/lib/x86_64-linux-gnu/:/home/guisoares/catkin_ws/install/lib/mavlink_sitl_gazebo/plugins
2022-07-06 12:21:58.224582: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-06 12:21:58.224606: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (guisoares-Aspire-A315-53): /proc/driver/nvidia/version does not exist
2022-07-06 12:21:58.225091: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv1D)              (None, 12796, 4)          464       
                                                                 
 batch1 (BatchNormalization)  (None, 12796, 4)         16        
                                                                 
 pool1 (AveragePooling1D)    (None, 6398, 4)           0         
                                                                 
 conv2 (Conv1D)              (None, 6393, 4)           100       
                                                                 
 batch2 (BatchNormalization)  (None, 6393, 4)          16        
                                                                 
 pool2 (AveragePooling1D)    (None, 6390, 4)           0         
                                                                 
 conv3 (Conv1D)              (None, 6387, 10)          1

In [5]:
opt = keras.optimizers.RMSprop(learning_rate=0.01)
# Create the loss function object using the wrapper function above
# spec_loss = specificity_loss_wrapper()
model.compile(
            loss=tf.keras.losses.BinaryCrossentropy(), 
            optimizer=opt, 
            metrics=['accuracy', tf.keras.metrics.SensitivityAtSpecificity(0.5)])

In [6]:
NEW_DATASET_PATH = "/media/guisoares/guisoares-ext-hdd/Datasets/chb-mit-segments/1"

train_files = glob.glob(os.path.join(NEW_DATASET_PATH,'train',"*.npz"))
test_files = glob.glob(os.path.join(NEW_DATASET_PATH,'test',"*.npz"))

print("Training files:")
print(train_files)
print("Testing files:")
print(test_files)

Training files:
['/media/guisoares/guisoares-ext-hdd/Datasets/chb-mit-segments/1/train/chb01_segments.npz', '/media/guisoares/guisoares-ext-hdd/Datasets/chb-mit-segments/1/train/chb02_segments.npz', '/media/guisoares/guisoares-ext-hdd/Datasets/chb-mit-segments/1/train/chb08_segments.npz']
Testing files:
['/media/guisoares/guisoares-ext-hdd/Datasets/chb-mit-segments/1/test/chb05_segments.npz']


In [7]:
# Load test arrays to memory
x_test = []
y_test = []
for file in test_files:
    test_arrays = np.load(file)
    x_test.append(test_arrays['arr_0'])
    y_test.append(test_arrays['arr_1'])

x_test = np.array(x_test)
x_test = x_test.reshape((x_test.shape[0]*x_test.shape[1], x_test.shape[2], x_test.shape[3]))
x_test = np.swapaxes(x_test, 1, 2)

y_test = np.array(y_test)
y_test = y_test.reshape((y_test.shape[0]*y_test.shape[1], -1))

In [8]:
def calculate_neg_pos(y):
    unique, counts = np.unique(y, return_counts=True)
    dic = dict(zip(unique, counts))
    neg = dic[0]
    pos = dic[1]
    return neg, pos

neg_test, pos_test = calculate_neg_pos(y_test)
print(f"Negatives: {neg_test}, Positives: {pos_test}")

Negatives: 9, Positives: 4


In [9]:
# callback = tf.keras.callbacks.EarlyStopping(monitor = 'accuracy', min_delta=0.05)

epochs = 10
for epoch in range(epochs):
    for file in train_files:

        # y = np.load('signals/chb01/labels.npy')
        # x = np.load('signals/chb01/signals.npy') 

        arrays = np.load(file)
        x_train = arrays['arr_0']
        y_train = arrays['arr_1']
        x_train = np.swapaxes(x_train, 1, 2) #swap axis: N,23,25600 -> N,25600, 23
        # x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, shuffle=True)

        print(f"Epoch: {epoch+1} ", end = "")
        unique, counts = np.unique(y_train, return_counts=True)
        print(f"Train :{dict(zip(unique, counts))} ", end="")

        history = model.fit(x_train, y_train, batch_size=32, epochs=1, validation_data=(x_test,y_test), verbose=2)

        if history.history['accuracy'][0] > 0.99:
            break
    if history.history['accuracy'][0] > 0.99:
        break
    
        # train_metrics = model.train_on_batch(x_train, y_train, return_dict=True)
        # test_metrics = model.test_on_batch(x_test, y_test, return_dict=True)

        # print(train_metrics, test_metrics)



Epoch: 1 Train :{0: 7, 1: 3} 1/1 - 5s - loss: 0.9618 - accuracy: 0.3000 - sensitivity_at_specificity: 0.3333 - val_loss: 2.5711 - val_accuracy: 0.3077 - val_sensitivity_at_specificity: 0.0000e+00 - 5s/epoch - 5s/step
Epoch: 1 Train :{0: 7, 1: 3} 1/1 - 0s - loss: 4.3758 - accuracy: 0.7000 - sensitivity_at_specificity: 1.0000 - val_loss: 1.4332 - val_accuracy: 0.3077 - val_sensitivity_at_specificity: 0.0000e+00 - 324ms/epoch - 324ms/step
Epoch: 1 Train :{0: 14, 1: 6} 1/1 - 0s - loss: 0.9722 - accuracy: 0.4500 - sensitivity_at_specificity: 0.8333 - val_loss: 2.5811 - val_accuracy: 0.3077 - val_sensitivity_at_specificity: 0.0000e+00 - 417ms/epoch - 417ms/step
Epoch: 2 Train :{0: 7, 1: 3} 1/1 - 0s - loss: 0.0175 - accuracy: 1.0000 - sensitivity_at_specificity: 1.0000 - val_loss: 2.1661 - val_accuracy: 0.3077 - val_sensitivity_at_specificity: 0.0000e+00 - 283ms/epoch - 283ms/step


In [10]:
history.history['accuracy']

[1.0]

In [11]:
y_pred = model.predict(x_train[:])
y_pred = np.where(y_pred < 0.5, 0, 1)
unique, counts = np.unique(y_pred, return_counts=True)
print(dict(zip(unique, counts)))

unique, counts = np.unique(y_train, return_counts=True)
print(dict(zip(unique, counts)))

1/1 [==============================] - 0s 360ms/step
{1: 10}
{0: 7, 1: 3}


In [12]:
import keras.backend as K
def compute_binary_specificity(y_pred, y_true):
    """Compute the confusion matrix for a set of predictions.

    Parameters
    ----------
    y_pred   : predicted values for a batch if samples (must be binary: 0 or 1)
    y_true   : correct values for the set of samples used (must be binary: 0 or 1)

    Returns
    -------
    out : the specificity
    """

    # check_binary(K.eval(y_true))    # must check that input values are 0 or 1
    # check_binary(K.eval(y_pred))    # 

    TN = np.logical_and(K.eval(y_true) == 0, K.eval(y_pred) == 0)
    FP = np.logical_and(K.eval(y_true) == 0, K.eval(y_pred) == 1)

    # as Keras Tensors
    TN = K.sum(K.variable(TN))
    FP = K.sum(K.variable(FP))

    specificity = TN / (TN + FP + K.epsilon())
    return specificity

def specificity_loss_wrapper():
    """A wrapper to create and return a function which computes the specificity loss, as (1 - specificity)

    """
    # Define the function for your loss
    def specificity_loss(y_true, y_pred):
        return 1.0 - compute_binary_specificity(y_true, y_pred)

    return specificity_loss    # we return this function object